In [12]:
# Data manipulation libraries
import pandas as pd
import numpy as np

# API 
from api import FED_API
import requests

# Machine learning libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Technical indicator library
#import talib as ta

# Data import library
import yfinance as yf

#Data visualisation
import plotly.graph_objs as go

#Data Source
import tradingeconomics as te

In [10]:
# Gather last 22yrs of data
SPY = yf.download('SPY', period = '50y', interval = '1wk')
NSD = yf.download('SPY', period = '50y', interval = '1wk')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
# Data does not go back far enough
SPY.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1993-01-25,43.96875,43.96875,43.75000,43.93750,25.627352,1003200.0
1993-02-01,43.96875,45.09375,43.96875,44.96875,26.228857,2234800.0
1993-02-08,44.96875,45.12500,44.53125,44.59375,26.010120,1159800.0
1993-02-15,44.46875,44.46875,42.81250,43.56250,25.408617,998700.0
1993-02-22,43.68750,44.43750,43.46875,44.40625,25.900763,1024300.0


In [16]:
# temp login to tradingeconomics
te.login()

'You are logged in as a guest:guest'

In [17]:
#check what indicators are avialable with Trading Economics API
te.getIndicatorData(output_type='df')

Without country indication only a list of available indicators will be returned...


,Category,CategoryGroup
0,Adp Employment Change,Labour
1,ANZ Roy Morgan Consumer Confidence Index,Consumer
2,API Crude Imports,Business
3,API Crude Oil Stock Change,Business
4,API Crude Runs,Business
5,API Cushing Number,Business
6,API Distillate Stocks,Business
7,API Gasoline Stocks,Business
8,API Heating Oil,Business
9,API Product Imports,Business


In [25]:
te.getIndicatorData(country='united states', output_type='df')

,Country,Category,Title,LatestValue,LatestValueDate,Source,Unit,URL,CategoryGroup,Adjustment,Frequency,HistoricalDataSymbol,CreateDate,PreviousValue,PreviousValueDate
0,United States,ADP Employment Change,United States ADP Employment Change,807.00,2021-12-31T00:00:00,"Automatic Data Processing, Inc.",Thousand,/united-states/adp-employment-change,Labour,SA,Monthly,UNITEDSTAADPEMPCHA,2014-04-30T13:57:00,505.00,2021-11-30T00:00:00
1,United States,API Crude Imports,United States API Crude Imports,-0.18,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-crude-imports,Business,,Weekly,UNITEDSTAAPICRUIMP,2020-07-20T10:54:00,0.11,2022-01-14T00:00:00
2,United States,API Crude Oil Stock Change,United States API Crude Oil Stock Change,-0.87,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-crude-oil-stock-change,Business,,Weekly,APICRUDEOIL,2017-11-28T15:21:00,1.40,2022-01-14T00:00:00
3,United States,API Crude Runs,United States API Crude Runs,0.01,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-crude-runs,Business,,Weekly,UNITEDSTAAPICRURUN,2020-07-20T11:02:00,-0.10,2022-01-14T00:00:00
4,United States,API Cushing Number,United States API Cushing Number,-1.04,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-cushing-number,Business,,Weekly,UNITEDSTAAPICUSNUM,2020-07-20T11:07:00,-1.50,2022-01-14T00:00:00
5,United States,API Distillate Stocks,United States API Distillate Stocks,-2.24,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-distillate-stocks,Business,,Weekly,UNITEDSTAAPIDISSTO,2020-07-20T11:09:00,-1.18,2022-01-14T00:00:00
6,United States,API Gasoline Stocks,United States API Gasoline Stocks,2.43,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-gasoline-stocks,Business,,Weekly,UNITEDSTAAPIGASSTO,2020-07-20T11:11:00,3.46,2022-01-14T00:00:00
7,United States,API Heating Oil,United States API Heating Oil,-0.49,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-heating-oil,Business,,Weekly,UNITEDSTAAPIHEAOIL,2020-07-20T11:12:00,-0.22,2022-01-14T00:00:00
8,United States,API Product Imports,United States API Product Imports,0.20,2022-01-21T00:00:00,American Petroleum Institute (API),BBL/1Million,/united-states/api-product-imports,Business,,Weekly,UNITEDSTAAPIPROIMP,2020-07-20T11:14:00,-0.09,2022-01-14T00:00:00
9,United States,Asylum Applications,United States Asylum Applications,191367.00,2020-12-31T00:00:00,US Department of Justice,Persons,/united-states/asylum-applications,Government,NSA,Yearly,UNITEDSTAASYAPP,2015-09-10T15:49:00,214249.00,2019-12-31T00:00:00


#### Finding: Not going to help

#### FED Info

In [24]:
fedsample_data = requests.get('https://api.stlouisfed.org/fred/series?series_id=GNPCA&api_key=FED_API&file_type=json')

In [25]:
print(fedsample_data)

<Response [400]>


In [27]:
base_url = "https://api.stlouisfed.org/fred/series?series_id=GNPCA&api_key="+FED_API+"&file_type=json"
fed_data = requests.get(base_url).json()

In [28]:
fed_data

{'realtime_start': '2022-01-27',
 'realtime_end': '2022-01-27',
 'seriess': [{'id': 'GNPCA',
   'realtime_start': '2022-01-27',
   'realtime_end': '2022-01-27',
   'title': 'Real Gross National Product',
   'observation_start': '1929-01-01',
   'observation_end': '2020-01-01',
   'frequency': 'Annual',
   'frequency_short': 'A',
   'units': 'Billions of Chained 2012 Dollars',
   'units_short': 'Bil. of Chn. 2012 $',
   'seasonal_adjustment': 'Not Seasonally Adjusted',
   'seasonal_adjustment_short': 'NSA',
   'last_updated': '2021-07-29 07:45:58-05',
   'popularity': 13,
   'notes': 'BEA Account Code: A001RX\n\n'}]}